In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [2]:
import pandas as pd

training_data = pd.read_csv('./Data/train.csv')
training_data.head(3)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1


In [3]:
duplicate_rows = training_data.duplicated(subset=['response_a', 'response_b'])
duplicate_rows.sum()

186

In [4]:
training_data = training_data.drop_duplicates(subset=['response_a', 'response_b'])

In [5]:
def determine_preference(row):
    if row['winner_model_a'] == 1:
        return 0
    elif row['winner_model_b'] == 1:
        return 1 
    elif row['winner_tie'] == 1:
        return 2
    else: #Invalid
        return -1 

training_data['preference'] = training_data.apply(determine_preference, axis=1)
(training_data['preference'] == -1).sum()

0

In [6]:
pd.set_option('display.max_colwidth', None)

training_data = training_data[['prompt', 'response_a', 'response_b', 'preference']]

In [7]:
training_data['prompt_len'] = training_data['prompt'].apply(len)
training_data = training_data.sort_values(by='prompt_len')
training_data.head(8)

,prompt,response_a,response_b,preference,prompt_len
4868,"[""abc""]","[""Hello! How can I assist you today?""]","[""Hello! It seems like you may have accidentally typed \""abc\"" instead of a question or a message. If you have a question or need assistance with something, feel free to let me know and I'll do my best to help you out.\n\nIf you don't have anything specific in mind, here's a fun fact about the alphabet: Did you know that the word \""alphabet\"" comes from the first two letters of the Greek alphabet, alpha and beta? The word \""alphabet\"" was first used in English in the 15th century, and it originally referred to the Greek alphabet specifically. It wasn't until later that the term was used to refer to any set of letters used to represent the sounds of a language.\n\nI hope you find that interesting! Let me know if you have any other questions or if there's anything else I can help you with.""]",2,7
34466,"[""Hey""]","[""Hello! How can I help you today? If you have any questions or need assistance with something, just let me know. I'm here to help.""]","[""Hello! How can I help you today? If you have any questions or need assistance with something, feel free to ask. I'm here to help.\n\nIf you're not sure where to start, here are a few things I can help you with:\n\n* Answering questions about a specific topic\n* Providing recommendations or suggestions\n* Generating ideas or brainstorming solutions\n* Helping you learn or understand a concept\n\nLet me know if you have something specific in mind, or if you'd like me to help with something else. I'm here to assist you in any way I can.\n\nIf you have any feedback or suggestions for how I can improve, I'd love to hear them. You can provide feedback by clicking on the \""Feedback\"" button in the lower right corner of the screen.\n\nI hope you have a great day! Let me know if you have any questions or need any help. I'm here for you.""]",1,7
44345,"[""hey""]","[""Hi there! What can I help you with?""]","[""Hello! How can I assist you today?""]",2,7
57316,"[""Rat""]","[""Rats are small rodents that are found all over the world. While they are often considered pests, they can also make great pets. If you have any specific questions or need information about rats, feel free to ask!""]","[""Rats are various medium-sized, long-tailed rodents found throughout the order Rodentia, with stereotypical rats belonging to the genus Rattus. They are typically slender with a pointed head, large eyes, and prominent, thinly furred ears. Rats have moderately long legs and long, sharp claws. Some common species of rats include:\n\n- Brown Rat (Rattus norvegicus): Also known as the Norway rat, it is a large member of the mouse family, averaging 16 inches (40 centimeters) in total length, including the tail.\n\n- Black Rat (Rattus rattus): This species is closely related to the brown rat and has played a significant role in the spread of diseases throughout history.\n\nRats are opportunistic survivors and often live with and near humans, making them known as commensals. They have been responsible for more human deaths than all wars and human conflicts combined, mainly due to the diseases they carry. However, rats also present benefits to their ecosystems as predators and prey, and their burrows help aerate the soil.\n\nIn human culture, the term \""rat\"" is used in criminal slang for an informant, and \""rat-like\"" usually implies an unattractive and suspicious person. Among trade unions, the word \""rat\"" is also a term for nonunion employers or breakers of union contracts. Rats have been featured in various works of fiction, often inaccurately portrayed.""]",1,7
41437,"[""sex""]","[""I'm here to provide helpful and respectful responses to your questions. If you have any specific questions or need information on a different topic, feel free to ask!""]","[""I'm sorry, but I'm unable to assist with that topic. Is there something else I can help you with?""]",2,7
14969,"[""hey""]","["

In [8]:
!pip install -U tokenizers;
!pip install -U transformers;
!pip install -U peft;
!pip install -U bitsandbytes;
!pip install huggingface_hub ipython "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git";

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-9w7oz8p4/unsloth_62f54573574f4418804ef6d95ad6459f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-9w7oz8p4/unsloth_62f54573574f4418804ef6d95ad6459f
  Resolved https://github.com/unslothai/unsloth.git to commit 4e570be9ae4ced8cdc64e498125708e34942befc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


  Using cached https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp39-cp39-manylinux2014_x86_64.whl (211.8 MB)
  Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
  Using cached pyarrow-17.0.0-cp39-cp39-manylinux_2_28_x86_64.whl (39.9 MB)


  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 12.0.1
    Uninstalling pyarrow-12.0.1:
      Successfully uninstalled pyarrow-12.0.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4


In [9]:
!pip install pyarrow==12.0.1 datasets==2.14.4

  Using cached pyarrow-12.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (39.0 MB)
  Using cached datasets-2.14.4-py3-none-any.whl (519 kB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.20.0
    Uninstalling datasets-2.20.0:
      Successfully uninstalled datasets-2.20.0


In [11]:
import torch
from datasets import Dataset
from unsloth import FastLanguageModel
from transformers import PreTrainedTokenizerFast, Trainer, TrainingArguments, EvalPrediction
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig
import numpy as np
from sklearn.metrics import log_loss, accuracy_score

class DataProcessor:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def split_dataset(self, training_data, subset_ratio=0.15):
        subset_data = training_data.sample(frac=subset_ratio, random_state=42)

        preference_dataset = Dataset.from_pandas(subset_data)
        train_temp_dataset = preference_dataset.train_test_split(test_size=0.2, seed=42)
        temp_dataset = train_temp_dataset['test'].train_test_split(test_size=0.5, seed=42)
        train_dataset = train_temp_dataset['train']
        val_dataset = temp_dataset['train']
        test_dataset = temp_dataset['test']
        return train_dataset, val_dataset, test_dataset

    def generate_prompt(self, data_point):
        # Special Tokens are missing          
        return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

        Prompt: {data_point['prompt']}
        Model A Response: {data_point['response_a']}
        Model B Response: {data_point['response_b']}
        """
            
    def preprocess(self, examples):
        return self.tokenizer(
            examples['prompt'],
            examples['response_a'],
            examples['response_b'],
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-08-01 17:12:26.295979: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-01 17:12:27.260763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
class ModelTrainer:
    def __init__(self, data_processor, config, num_labels=3):
        self.data_processor = data_processor

        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name=config['checkpoint'],
            dtype=torch.float16,
            load_in_4bit=True,
            device_map="auto"
        )

        self.model.lm_head = torch.nn.Linear(in_features=self.model.lm_head.in_features, out_features=num_labels, bias=False)

        self.freeze_model_layers(config)

        self.model = prepare_model_for_kbit_training(self.model)
        
        lora_config = LoraConfig(
            r=config["lora_r"],
            lora_alpha=config["lora_alpha"],
            lora_dropout=config["lora_dropout"],
            bias=config["lora_bias"]
        )

        self.model = get_peft_model(self.model, lora_config)
        self.model.gradient_checkpointing_enable()

        self.training_args = TrainingArguments(
            output_dir=config["output_dir"],
            evaluation_strategy="epoch",
            per_device_train_batch_size=config["per_device_train_batch_size"],
            per_device_eval_batch_size=config["per_device_eval_batch_size"],
            num_train_epochs=config["n_epochs"],
            warmup_steps=config["warmup_steps"],
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=10,
            fp16=True,
            gradient_accumulation_steps=2,
            dataloader_num_workers=8
        )

        self.trainer = Trainer(
            model=self.model,
            args=self.training_args,
            tokenizer=self.tokenizer,
            compute_metrics=self.compute_metrics,
        )

        self.model.to(torch.device('cuda'))

    def freeze_model_layers(self, config):
        num_layers = len(self.model.model.layers)
        layers_to_freeze = min(config['freeze_layers'], num_layers)
        for i in range(layers_to_freeze):
            for param in self.model.model.layers[i].parameters():
                param.requires_grad = False

    def compute_metrics(self, eval_preds: EvalPrediction):
        preds = eval_preds.predictions.argmax(-1)
        labels = eval_preds.label_ids
        probs = torch.from_numpy(eval_preds.predictions).float().softmax(-1).numpy()

        loss = log_loss(y_true=labels, y_pred=probs)
        acc = accuracy_score(y_true=labels, y_pred=preds)
        return {"acc": acc, "log_loss": loss}

    def prepare_datasets(self, train_data, val_data, test_data):
        train_dataset = train_data.map(self.generate_prompt_for_example)
        val_dataset = val_data.map(self.generate_prompt_for_example)
        test_dataset = test_data.map(self.generate_prompt_for_example)

        train_dataset = self.process_dataset(train_dataset)
        val_dataset = self.process_dataset(val_dataset)
        test_dataset = self.process_dataset(test_dataset)

        return train_dataset, val_dataset, test_dataset

    def generate_prompt_for_example(self, example):
        generated_prompt = self.data_processor.generate_prompt(example)
        example['prompt'] = generated_prompt
        return example

    def process_dataset(self, dataset):
        dataset = dataset.map(lambda examples: {
            **self.data_processor.preprocess(examples), 
            'labels': examples['preference']
        }, batched=True)
        
        return dataset

    def train_model(self, train_dataset, val_dataset):
        def preprocess_and_add_labels(examples):
            tokenized = self.data_processor.preprocess(examples)
            labels = examples['preference']
            max_length = tokenized['input_ids'].shape[1]
            labels_padded = torch.full((len(labels), max_length), -100, dtype=torch.long)
            labels_padded[:, :len(labels)] = torch.tensor(labels).unsqueeze(1)

            return {**tokenized, 'labels': labels_padded.to('cuda')}

        train_dataset = train_dataset.map(preprocess_and_add_labels, batched=True)
        val_dataset = val_dataset.map(preprocess_and_add_labels, batched=True)

        def to_device(batch):
            for k, v in batch.items():
                if isinstance(v, torch.Tensor):
                    batch[k] = v.to('cuda')
            return batch

        train_dataset = train_dataset.map(to_device, batched=True)
        val_dataset = val_dataset.map(to_device, batched=True)

        self.trainer.train_dataset = train_dataset
        self.trainer.eval_dataset = val_dataset

        self.trainer.train()

In [13]:
config = {
    "checkpoint": "unsloth/llama-2-7b-bnb-4bit",
    "n_epochs": 1,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 4,
    "freeze_layers": 24, 
    "warmup_steps": 15,
    "lora_r": 4,
    "lora_alpha": 8,
    "lora_dropout": 0.05,
    "lora_bias": "none",
}

tokenizer = PreTrainedTokenizerFast.from_pretrained(config['checkpoint'])
data_processor = DataProcessor(tokenizer)

In [14]:
model_trainer = ModelTrainer(data_processor, config)
model_trainer.model

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA A16. Max memory: 14.542 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.22.post7. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/opt/conda/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


PeftModel(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear4bit(in_fea

In [15]:
train_dataset, val_dataset, test_dataset = data_processor.split_dataset(training_data)

In [16]:
train_dataset_with_prompt, val_dataset_with_prompt, test_dataset_with_prompt = model_trainer.prepare_datasets(train_dataset, val_dataset, test_dataset)

Map:   0%|          | 0/6875 [00:00<?, ? examples/s]

Map:   0%|          | 0/859 [00:00<?, ? examples/s]

Map:   0%|          | 0/860 [00:00<?, ? examples/s]

Map:   0%|          | 0/6875 [00:00<?, ? examples/s]

Map:   0%|          | 0/859 [00:00<?, ? examples/s]

Map:   0%|          | 0/860 [00:00<?, ? examples/s]

In [17]:
# fine-tuning with prompt
model_trainer.train_model(train_dataset_with_prompt, val_dataset_with_prompt)

Map:   0%|          | 0/6875 [00:00<?, ? examples/s]

Map:   0%|          | 0/859 [00:00<?, ? examples/s]

Map:   0%|          | 0/6875 [00:00<?, ? examples/s]

Map:   0%|          | 0/859 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,875 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 859
 "-____-"     Number of trainable parameters = 2,097,152
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just 

Epoch,Training Loss,Validation Loss


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacty of 14.54 GiB of which 193.00 MiB is free. Process 2126901 has 14.35 GiB memory in use. Of the allocated memory 14.00 GiB is allocated by PyTorch, and 267.15 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [18]:
torch.save(model_trainer.model.state_dict(), './Llama/Llama_prompt/Llama_state_dict.pth')
# Save the model and tokenizer
model_trainer.model.save_pretrained('./Llama/Llama_prompt')
model_trainer.tokenizer.save_pretrained('./Llama/Llama_prompt')

('./Llama/Llama_prompt/tokenizer_config.json',
 './Llama/Llama_prompt/special_tokens_map.json',
 './Llama/Llama_prompt/tokenizer.model',
 './Llama/Llama_prompt/added_tokens.json',
 './Llama/Llama_prompt/tokenizer.json')

In [40]:
test_dataset_with_prompt.data['prompt'][0]

<pyarrow.StringScalar: 'Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred response based on the human preference label (Preference). Return the predicted preference as one of three labels: \'winner_model_a\', \'winner_model_b\', or \'winner_tie\', along with the logits for each label.\n\n        Prompt: ["Rate the following sentence on a scale of positive to negative sentiment, where 5 is the most positive, and 1 is the most negative. The sentence is: \\"The economy will definitely grow at a faster pace next year than this year.\\""]\n        Model A Response: ["5 (Most Positive)\\n\\nExplanation: The sentence conveys a positive sentiment about the economy\'s growth, which is rated as 5 on the scale. The use of the word \\"definitely\\" adds to the certainty and confidence in the statement. This is a clear indication that the economy will 

In [44]:
test_dataset_with_prompt.data['labels'][0]

<pyarrow.Int64Scalar: 0>

In [46]:
torch.cuda.device_count()

1